# Building a Datapane Dashboard

Code and data from <a href="https://towardsdatascience.com/building-an-interactive-python-dashboard-using-sql-and-datapane-46bd92294fd3">Building an interactive Python dashboard using SQL and Datapane</a> - John Micah Reid, June 2021

We'll be working with a modified version (`chinook_m.db`) of a database called [Chinook](https://github.com/lerocha/chinook-database). The Chinook database contains information about a fictional digital music shop.

The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables.

The Chinook database is provided as a SQLite database file called `chinook_m.db`. It's worth remembering that our database retains 'state', so if we run a query with a `CREATE` or `DROP` twice, the query will fail. 

<b>[Marcos] Note for this seminar/homework:</b> if the database gets locked, there's a `chinook.db` file on GitHub that can be copied over the `chinook_m.db` to restore it back to its initial state.

Here's a schema diagram for the Chinook database:


In [ ]:
from IPython.display import Image
Image(filename='Chinook_m.png', width="600") 

In [ ]:
# installing the Datapane library
!pip install datapane

In [ ]:
# Import the necessary libraries
import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
# remember to install the Datapane library (see Preparation_BeforeClass on GitHub)
import datapane as dp

In [ ]:
# The database we will be using
# IMPORTANT: adjust the path to reflect the location of your file!
db = './chinook_m.db'

In [ ]:
# Main function for running SQL queries
def run_query(q):
    # Connect to the database
    with sqlite3.connect(db) as conn:
        # Wrapper function to read_sql_query(): returns a DataFrame corresponding to the result set of 
        # the query string
        return pd.read_sql(q,conn)
    
#def run_command(c):
#    with sqlite3.connect(db) as conn:
#        conn.isolation_level = None
#        conn.execute(c) 

`sql_master` is the <b>schema table</b>: it contains a description of all of the other tables, indexes, triggers, and views that are contained within the database.

Have a look <a href="https://www.sqlite.org/schematab.html">here</a> for a complete description.

In [ ]:
# Function to show all the tables present in the database.
def show_tables():
    q = '''
        SELECT
            name
        FROM sqlite_master
        WHERE type IN ("table","view");
        '''
    return run_query(q)

# Function to count how many rows in each table
def get_table_row_count(tablename):
    q = '''
        SELECT
            COUNT(1)
        FROM %s;
        ''' % tablename
    return run_query(q)["COUNT(1)"][0]

tables = show_tables()
tables["row_count"] = [get_table_row_count(t) for t in tables["name"]]

tables

### IMPORTANT: connect to <a href="https://datapane.com/">Datapane</a> using your API token

#### See [Preparation_BeforeClass](https://github.com/lse-st207/lectures2021/tree/main/Preparation_BeforeSeminar) on GitHub for how to register and get an API token.

In [ ]:
# Uncomment the following line and put your Datapane token
#!datapane login --token=YOUR_TOKEN_HERE

In [ ]:
# Building a first datapane
import datapane as dp 

# Query: the best-selling tracks
# We start by selecting the columns we want to see in the output. Since these come from four different tables 
# (track, album, artist, invoice_line), we need to join the tables on the foreign key relationships — check the 
# schema to see what is possible. Next we group by the track name, order by the most purchases and only select 
# the top 10 results.

top_10_tracks_query = '''
SELECT 
    t.name trackname,
    a.title album_title,
    ar.name artist,
    count(*) as total_purchases,
    sum(il.unit_price) total_cost
FROM track t 
JOIN album a on a.album_id = t.album_id
JOIN artist ar on ar.artist_id = a.artist_id
JOIN invoice_line il on il.track_id = t.track_id
GROUP BY 1
ORDER BY total_purchases desc
LIMIT 10
'''

top_10_df = run_query(top_10_tracks_query)

table = dp.Table(top_10_df)
report = dp.Report(table)
report.upload(name='Music Sales')

## Selecting Albums to Purchase

In [ ]:
# Query: tracks sold by genre
albums_to_purchase = '''
WITH 
    usa_tracks_sold AS
        (
         SELECT il.* FROM invoice_line il
         INNER JOIN invoice i ON il.invoice_id = i.invoice_id
         INNER JOIN customer c ON i.customer_id = c.customer_id
        )
SELECT 
    g.name Genre, 
    COUNT(uts.invoice_line_id) "Number of Tracks Sold", 
    CAST(COUNT(uts.invoice_line_id) AS FLOAT)/(SELECT COUNT(*) FROM usa_tracks_sold) "Percentage of Tracks Sold"
FROM usa_tracks_sold uts
INNER JOIN track t ON uts.track_id = t.track_id
INNER JOIN genre g ON t.genre_id = g.genre_id
GROUP BY 1 
ORDER BY 2 DESC
LIMIT 10;
'''

genre_sales = run_query(albums_to_purchase)
genre_sales.sort_values('Number of Tracks Sold', inplace=True)

genre_sales_plot = px.bar(
    genre_sales, 
    x="Percentage of Tracks Sold", 
    y="Genre", 
    orientation='h',
    text="Percentage of Tracks Sold",
    title = "Sales by Genre"
)

genre_sales_plot.update_traces(texttemplate='%{text:.1%}', textposition='outside')
genre_sales_plot.show()

### Observations

Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following artists:

- Red Tone (Punk)
- Slim Jim Bites (Blues)
- Meteor and the Girls (Pop)

It's worth keeping in mind that combined, these three genres only make up only 17% of total sales, so we should be on the lookout for artists and albums from the 'rock' genre, which accounts for 53% of sales.

## Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [ ]:
# Query: Who are our best-performing sales employees, and when do they make the most sales?

# We use strftime to truncate the timestamps so we can group by month. This makes the data 
# look nicer when we plot it.
# We use a subquery (also called a Common Table Expression) to calculate an intermediate 
# table (customer_support_rep_sales). We then call this table in the final query.

employee_sales_performance = '''
WITH 
    customer_support_rep_sales AS
        (
         SELECT 
             i.customer_id,
             strftime('%Y-%m', i.invoice_date) || "-01" Month,
             c.support_rep_id,
             SUM(i.total) Sales_Total
         FROM invoice i
         INNER JOIN customer c ON c.customer_id = i.customer_id
         GROUP BY 2, 3
        )
SELECT
    e.first_name || " " || e.last_name "Employee Name",
    csrs.Month,
    SUM(csrs.Sales_total) "Amount of Sales (in $)"
FROM customer_support_rep_sales csrs
INNER JOIN employee e ON csrs.support_rep_id = e.employee_id
GROUP BY 1, 2;
'''

run_query(employee_sales_performance)

In [ ]:
# We did a groupby in Pandas to remove the ‘Month’ column for the second chart.
employee_sales = run_query(employee_sales_performance)
employee_sales["Month"] = pd.to_datetime(employee_sales["Month"])

employee_sales_monthly = px.line(employee_sales,
                            x = "Month",
                            y = "Amount of Sales (in $)",
                            color = "Employee Name")

employee_sales_monthly.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

employee_sales_monthly.show()

In [ ]:
employee_sales_total = employee_sales.groupby(["Employee Name"]) \
                                        .sum() \
                                        .sort_values(by="Amount of Sales (in $)") \
                                        .reset_index()

employee_sales_bar = px.bar(employee_sales_total, 
                            y = 'Employee Name', 
                            x = "Amount of Sales (in $)",
                            text = "Amount of Sales (in $)",
                            orientation = 'h')

employee_sales_bar.update_traces(texttemplate='%{text:.1f}')
employee_sales_bar.show()

In [ ]:
# Generating the datapane
dp.Report(
    dp.Plot(employee_sales_monthly, name = "employee-sales-monthly"),
    dp.Plot(employee_sales_bar, name = "employee-sales-bar"),
).upload(name="Employee Sales")

### Observations

While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

## Analyzing Sales by Country

Our next task is to analyze the sales data for customers from each different country. We have been given guidance to use the country value from the `customers` table, and ignore the country from the billing address in the `invoice` table.

In particular, you have been directed to calculate data, for each country, on the:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Because there are a number of countries with only one customer, we should group these customers as "Other" in our analysis.

In [ ]:
sales_by_country = '''
WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )


SELECT
    country,
    count(distinct customer_id) customers,
    SUM(unit_price) total_sales,
    SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
    SUM(unit_price) / count(distinct invoice_id) average_order,
    CASE
        WHEN country = "Other" THEN 1
        ELSE 0
    END AS sort
FROM country_or_other
GROUP BY country
ORDER BY sort ASC, total_sales DESC;
'''

run_query(sales_by_country)

## Visualizing Sales by Country

Now that we have our data, we've been asked to create a series of visualizations which communicate our findings, and then make recommendations on which countries may have potential for growth, so the Chinook marketing team can create some new advertising campaigns.

In [ ]:
country_metrics = run_query(sales_by_country)

sales_breakdown = go.Figure(data=[go.Pie(
    labels = country_metrics["country"], 
    values=country_metrics["total_sales"], 
    hole=.3)
                                 ])

sales_breakdown.update_layout(title_text="Sales by country")
sales_breakdown.show()

In [ ]:
customer_ratio = country_metrics["customers"] / country_metrics["customers"].sum() * 100.0
sales_ratio = country_metrics["total_sales"] / country_metrics["total_sales"].sum() * 100.0


custs_vs_dollars = go.Figure(data=[
    go.Bar(name='Customers', x=country_metrics["country"], y=customer_ratio),
    go.Bar(name='Total Sales', x=country_metrics["country"], y=sales_ratio),
])

# Change the bar mode
custs_vs_dollars.update_layout(barmode='group', title_text="Pct customers vs sales")
custs_vs_dollars.show()

In [ ]:
country_metrics["difference_from_avg"] = (country_metrics["average_order"] - \
                                          country_metrics["average_order"].mean()) * 100.0

avg_order = px.bar(
    country_metrics, 
    x = "country", 
    y = "difference_from_avg", 
    color = "country", 
    title = "Avg order - Pct difference from mean",
    labels={
        "country": "Country",
        "difference_from_avg": "Difference from mean (%)"
    }
)

avg_order.update_layout(showlegend=False)
avg_order.show()

In [ ]:
clv = px.bar(
    country_metrics,
    x = 'country',
    y = 'customer_lifetime_value',
    color='country',
    labels={
        "country": "Country",
        "customer_lifetime_value": "Customer Lifetime Value ($)"
    },
    title = "Customer Lifetime Value"
)

clv.update_layout(showlegend=False)
clv.show()

In [ ]:
customers = tables[tables["name"]=="customer"]["row_count"].values[0]
employees = tables[tables["name"]=="employee"]["row_count"].values[0]
tracks_sold = tables[tables["name"]=="invoice_line"]["row_count"].values[0]

In [ ]:
dp.Report(
    dp.Group(
        dp.BigNumber(heading="Total customers", value = customers),
        dp.BigNumber(heading="Total employees", value = employees),
        dp.BigNumber(heading="Total tracks sold", value = tracks_sold),
        columns = 3
    ),
    dp.Plot(genre_sales_plot, name = "genre_sales"),
    dp.Plot(employee_sales_monthly, name = "employee-sales-1"),
    dp.Plot(employee_sales_bar, name = "employee-sales-2"),
    dp.Group(
        dp.Plot(sales_breakdown),
        dp.Plot(custs_vs_dollars),
        dp.Plot(avg_order),
        dp.Plot(clv),
        columns=2
    ),
    dp.DataTable(top_10_df, name = "track-sales"),
).upload(name="Interactive Dashboard using SQLite")

### WEEK 2 - HOMEWORK B

Write additional SQL commands for the following questions:

1. Which tracks are on the playlist "Brazilian Music"?
2. How many tracks are there by each of Amy Winehouse's albums? (tip: you can use some aggregation function to count the number of tracks).

Go back to the `# Building a first datapane` cell and use the same structure here. Design your SQL query and show the results in a Datapane. Do this for each question.